<a href="https://colab.research.google.com/github/manhtiendoan/CSC4850-Machine-Learning-Project/blob/main/OKC_Starsign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%matplotlib inline

import numpy as np
import pandas as pd
import sklearn 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
df = pd.read_csv('drive/MyDrive/okcupid_profiles.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   status       59946 non-null  object 
 2   sex          59946 non-null  object 
 3   orientation  59946 non-null  object 
 4   body_type    54650 non-null  object 
 5   diet         35551 non-null  object 
 6   drinks       56961 non-null  object 
 7   drugs        45866 non-null  object 
 8   education    53318 non-null  object 
 9   ethnicity    54266 non-null  object 
 10  height       59943 non-null  float64
 11  income       59946 non-null  int64  
 12  job          51748 non-null  object 
 13  last_online  59946 non-null  object 
 14  location     59946 non-null  object 
 15  offspring    24385 non-null  object 
 16  pets         40025 non-null  object 
 17  religion     39720 non-null  object 
 18  sign         48890 non-null  object 
 19  smok

In [32]:
pd.set_option("display.max_columns", None)
df.head()


,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,last_online,location,offspring,pets,religion,sign,smokes,speaks,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75.0,-1,transportation,2012-06-28-20-30,"south san francisco, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism and very serious about it,gemini,sometimes,english,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,70.0,80000,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn't have kids, but might want them",likes dogs and likes cats,agnosticism but not too serious about it,cancer,no,"english (fluently), spanish (poorly), french (...",i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,68.0,-1,NaN,2012-06-27-09-10,"san francisco, california",NaN,has cats,NaN,pisces but it doesn&rsquo;t matter,no,"english, french, c++","i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,71.0,20000,student,2012-06-28-14-22,"berkeley, california",doesn't want kids,likes cats,NaN,pisces,no,"english, german (poorly)",i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",66.0,-1,artistic / musical / writer,2012-06-27-21-26,"san francisco, california",NaN,likes dogs and likes cats,NaN,aquarius,no,english,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN


# Data cleaning/splicing checklist:
useful resource maybe:
https://machinelearningmastery.com/one-hot-encoding-for-categorical-data/
 * 0   age          - no work needed
 * 1   status       - clean, needs converting to numeric 
 * 2   sex          - clean, needs converting to numeric (0 male, 1 female?)
 * 3   orientation  - clean, needs converting to numeric
 * 4   body_type    - clean, needs converting to numeric (perhaps based on BMI?) 
 * 5   diet         - clean, needs converting to numeric
 * 6   drinks       - clean, needs converting to numeric (this one is sort of linear)
 * 7   drugs        - clean, needs converting to numeric (same as drinks)
 * 8   education    - clean, needs converting to numeric (sort of linear again? maybe increasing based on level of education?) 
 * 9   ethnicity    - too many labels (multi-racial profiles), maybe needs converting into multiple boolean columns
 * 10  height       - no work needed
 * 11  income       - no work needed  
 * 12  job          - surprisingly clean, OKCupid must've forced people to choose these from a list
 * 13  last_online  - clean, maybe convert to a single time measure like seconds or days since 2000?)
 * 14  location     - not super clean, need to figure out
 * 15  offspring    - not super clean, need to figure out 
 * 16  pets         - splicable, need to figure out 
 * 17  religion     - can be spliced into 'religion' and 'religion attitude'?, otherwise clean and needs converteing to numeric
 * 18  sign         - can be spliced into 'sign' and 'sign_attitude'?, otherwise clean and needs converteing to numeric 
 * 19  smokes       - clean, needs converting to numeric (same as drinks)
 * 20  speaks       - same as ethnicity, need way to deal with multi-lingual profiles

 ***
 # Henceforth is an absolute mess, needs discussion
 * 21  essay0       54458 non-null  object 
 * 22  essay1       52374 non-null  object 
 * 23  essay2       50308 non-null  object 
 * 24  essay3       48470 non-null  object 
 * 25  essay4       49409 non-null  object 
 * 26  essay5       49096 non-null  object 
 * 27  essay6       46175 non-null  object 
 * 28  essay7       47495 non-null  object 
 * 29  essay8       40721 non-null  object 
 * 30  essay9       47343 non-null  object




In [ ]:
#Finding unique values for all columns of data
column_names = list(df.columns.values)
#print(column_names)

for x in column_names[:21]:
  print(x + ": " + str(df[x].unique()))

In [ ]:
#Found a way to rank order word frequencies 
from collections import Counter
Counter(df['essay0'][0].split()).most_common(100)

